In [ ]:
%pip install peft transformers trl==0.11.0 evaluate torch sacrebleu
%pip install -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolve

In [ ]:
from dataclasses import (
    dataclass,
    field
)
from typing import Optional
import torch
from accelerate import Accelerator
from datasets import (
    load_dataset,
    DatasetDict
)
from peft import (
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)
from tqdm import tqdm
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    Adafactor,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    set_seed,
    GenerationConfig,
    EarlyStoppingCallback
)
from trl import (
    AutoModelForSeq2SeqLMWithValueHead,
    create_reference_model,
    PPOConfig,
    PPOTrainer
)
from trl.core import LengthSampler
import pandas as pd
import evaluate
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import csv

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

In [ ]:
tqdm.pandas()

In [ ]:
!unzip reward_model.zip

Archive:  reward_model.zip
   creating: content/reward_model/
  inflating: content/reward_model/adapter_config.json  
  inflating: content/reward_model/adapter_model.safetensors  
   creating: content/reward_model/tokenizer/
  inflating: content/reward_model/tokenizer/tokenizer_config.json  
  inflating: content/reward_model/tokenizer/vocab.txt  
  inflating: content/reward_model/tokenizer/special_tokens_map.json  
  inflating: content/reward_model/tokenizer/tokenizer.json  
   creating: content/reward_model/checkpoint-11/
  inflating: content/reward_model/checkpoint-11/training_args.bin  
  inflating: content/reward_model/checkpoint-11/tokenizer_config.json  
  inflating: content/reward_model/checkpoint-11/optimizer.pt  
  inflating: content/reward_model/checkpoint-11/vocab.txt  
  inflating: content/reward_model/checkpoint-11/trainer_state.json  
  inflating: content/reward_model/checkpoint-11/adapter_config.json  
  inflating: content/reward_model/checkpoint-11/adapter_model.safeten

In [ ]:
!unzip flan-t5-xl-best.zip

Archive:  flan-t5-xl-best.zip
   creating: flan-t5-xl-best/
  inflating: flan-t5-xl-best/tokenizer_config.json  
  inflating: flan-t5-xl-best/adapter_model.safetensors  
  inflating: flan-t5-xl-best/README.md  
  inflating: flan-t5-xl-best/tokenizer.json  
  inflating: flan-t5-xl-best/spiece.model  
  inflating: flan-t5-xl-best/adapter_config.json  
  inflating: flan-t5-xl-best/training_args.bin  
  inflating: flan-t5-xl-best/special_tokens_map.json  


In [ ]:
# reward_model_name = 'content/reward_model/checkpoint-44'
reward_model_name = 'content/reward_model'
reward_tokenizer_name = 'content/reward_model/tokenizer'
model_name = 'flan-t5-xl-best'
dataset_name = 'rest_flan-t5-xl_len4.csv'

In [ ]:
flan_tokenizer = AutoTokenizer.from_pretrained(model_name)
reward_tokenizer = AutoTokenizer.from_pretrained(reward_tokenizer_name)

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
if torch.cuda.is_available():
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map='auto', quantization_config=quantization_config)
    print('loaded GPU')
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

loaded GPU


In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    inference_mode=False,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 9,437,184 || all params: 2,859,194,368 || trainable%: 0.3301


In [ ]:
if torch.cuda.is_available():
  model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(model, device_map='auto')

In [ ]:
ref_model = create_reference_model(model)
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
typical_fields = ["max_position_embeddings", "n_positions", "seq_len", "seq_length", "n_ctx", "sliding_window"]

# Check which attribute a given model object has
context_windows = [getattr(model.config, field) for field in typical_fields if field in dir(model.config)]

# Grab the last one in the list; usually there's only 1 anyway
max_length = (context_windows.pop()) if len(context_windows) else print(f"No Max input variable found for {model_name}")

print(max_length)

512


In [ ]:
dataset = load_dataset("csv", data_files=dataset_name, split="train")

ds_train_devtest = dataset.train_test_split(test_size=24, seed=42)

dataset_splits = DatasetDict({
    'train': ds_train_devtest['train'],
    'test': ds_train_devtest['test'],
})

print("Before:\n", dataset)
print("After:\n", dataset_splits)

dataset_splits['test'].to_csv('reward_test_dataset.csv')

print(dataset_splits['train'])

Generating train split: 0 examples [00:00, ? examples/s]

Before:
 Dataset({
    features: ['Unnamed: 0', 'Source', 'Target', 'Gen_comp', 'ChrF', 'Cos_sim_t5', 'Gen_comp1', 'Gen_comp2', 'Total_ChrF'],
    num_rows: 248
})
After:
 DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Source', 'Target', 'Gen_comp', 'ChrF', 'Cos_sim_t5', 'Gen_comp1', 'Gen_comp2', 'Total_ChrF'],
        num_rows: 224
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Source', 'Target', 'Gen_comp', 'ChrF', 'Cos_sim_t5', 'Gen_comp1', 'Gen_comp2', 'Total_ChrF'],
        num_rows: 24
    })
})


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['Unnamed: 0', 'Source', 'Target', 'Gen_comp', 'ChrF', 'Cos_sim_t5', 'Gen_comp1', 'Gen_comp2', 'Total_ChrF'],
    num_rows: 224
})


In [ ]:
def build_dataset(
    tokenizer,
    dataset,
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """

    num_proc = 24

    def preprocess_function(examples):
        new_examples = {
            "query": [],
            "input_ids": [],
        }
        for source in examples["Source"]:
            query = source
            tokenized_prompt = tokenizer(query, truncation=True)
            new_examples["query"].append(query)
            new_examples["input_ids"].append(tokenized_prompt["input_ids"])

        return new_examples

    ds = dataset.map(
        preprocess_function,
        batched=True,
        num_proc=num_proc,
        remove_columns=dataset_splits['train'].column_names,
    )
    ds = ds.filter(lambda x: len(x["input_ids"]) < max_length, batched=False, num_proc=num_proc)

    ds.set_format(type="torch")
    return ds

In [ ]:
encoded_dataset = build_dataset(flan_tokenizer, dataset_splits['train'])

print(encoded_dataset)

encoded_testset = build_dataset(flan_tokenizer, dataset_splits['test'])

print(encoded_testset)

def collator(data):
    return {key: [d[key] for d in data] for key in data[0]}

Map (num_proc=24):   0%|          | 0/224 [00:00<?, ? examples/s]

Filter (num_proc=24):   0%|          | 0/224 [00:00<?, ? examples/s]

Dataset({
    features: ['query', 'input_ids'],
    num_rows: 224
})


Map (num_proc=24):   0%|          | 0/24 [00:00<?, ? examples/s]

Filter (num_proc=24):   0%|          | 0/24 [00:00<?, ? examples/s]

Dataset({
    features: ['query', 'input_ids'],
    num_rows: 24
})


In [ ]:
ppo_config = PPOConfig(
    '/rl/ppoconfig',
    log_with="wandb",
    learning_rate=1e-5,
    batch_size=16,
    mini_batch_size=1,
    gradient_accumulation_steps=2,
    seed=42,
    init_kl_coef=0.05,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [ ]:
set_seed(42)

In [ ]:
ppo_trainer = PPOTrainer(
    config=ppo_config,
    model=model,
    ref_model=ref_model,
    tokenizer=flan_tokenizer,
    dataset=encoded_dataset,
    data_collator=collator,
)

/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
sentiment_pipe = pipeline("sentiment-analysis", model=reward_model, tokenizer=reward_tokenizer)

Device set to use cuda:0


In [ ]:
sent_kwargs = {"top_k": None, "function_to_apply": "none", "return_all_scores": True, "batch_size": 1}

In [ ]:
generation_kwargs = {
    "top_k": 0.0,
    "do_sample": True,
    "pad_token_id": flan_tokenizer.pad_token_id,
}

output_min_length = 1
output_max_length = max_length
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    question_tensors = batch["input_ids"]

    response_tensors = ppo_trainer.generate(
        question_tensors,
        return_prompt=False,
        length_sampler=output_length_sampler,
        **generation_kwargs,
    )
    batch["response"] = flan_tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

    # Compute reward score (using the sentiment analysis pipeline)
    rewards = []
    for i in range(len(batch['query'])):
        texts = batch['query'][i] + '\n' + batch['response'][i]
        pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
        rewards.append(torch.tensor([output["score"] for output in pipe_outputs][0]))

    # Run PPO step
    stats = ppo_trainer.step(question_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
1it [00:43, 43.42s/it]/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more

IndexError: index -1 is out of bounds for dimension 0 with size 0

In [ ]:
def con_sent_emb(gen_sen: list[str], tar_sen: list[str]) -> tuple[any, any]:
    """
    Function to retrieve the contextualised sentence embeddings
    Args:
         gen_sen: A list of lists of strings
         tar_sen: A list of lists of strings
    Returns:
        A list of lists of contextual sentence embeddings
    """
    senTran = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")  # "sentence-transformers/sentence-t5-base"
    #model = SentenceTransformer("all-MiniLM-L6-v2")  # SentenceBERT

    # Generate the contextual embeddings for the generated and the target completions
    gen_emb = senTran.encode(gen_sen)  # , convert_to_tensor=True
    tar_emb = senTran.encode(tar_sen)  # , convert_to_tensor=True

    return gen_emb, tar_emb

In [ ]:
#### get results from test set
game_data = dict()
game_data['query'] = []
game_data['target'] = []
game_data["response (before)"] = []
game_data["response (after)"] = []
game_data['reward (before)'] = []
game_data['reward (after)'] = []
game_data['chrf (before)'] = []
game_data['chrf (after)'] = []
game_data['chrf difference'] = []
game_data['cos_sim (before)'] = []
game_data['cos_sim (after)'] = []
game_data['cos_sim difference'] = []

dataset_splits['test'].set_format("pandas")
df_test = encoded_testset
game_data["query"] = df_test["query"]
query_tensors = df_test["input_ids"]
device='cuda' if torch.cuda.is_available() else 'cpu'

response_tensors = []

#### get response from policy and ref model
for i in range(len(encoded_testset)):
    query = query_tensors[i].to(device).unsqueeze(0)

    query_response = model.generate(
        input_ids=query,
        **generation_kwargs
    ).squeeze()
    response_len = len(query_response) - len(query)
    response_tensors.append(query_response[-response_len:])

    #### decode responses
    game_data["response (before)"].append(dataset_splits['test']['Gen_comp'][i])

    game_data["response (after)"].append(flan_tokenizer.decode(response_tensors[i], skip_special_tokens=True))

    #### sentiment analysis of query/response pairs before/after
    text_before = game_data['query'][i] + '\n' + game_data['response (before)'][i]
    output_before = sentiment_pipe(text_before, **sent_kwargs)
    reward_before = [output["score"] for output in output_before][0]

    text_after = game_data['query'][i] + '\n' + game_data['response (after)'][i]
    output_after = sentiment_pipe(text_after, **sent_kwargs)
    reward_after = [output["score"] for output in output_after][0]

    game_data['reward (before)'].append(reward_before)
    game_data['reward (after)'].append(reward_after)

    # ChrF score
    chrf = evaluate.load('chrf')
    target = dataset_splits['test']['Target'][i]

    chrf_before = dataset_splits['test']['ChrF'][i]

    chrf_after = chrf.compute(predictions=[game_data["response (after)"][i]], references=[target])['score']

    game_data['target'].append(target)
    game_data['chrf (before)'].append(chrf_before)
    game_data['chrf (after)'].append(chrf_after)
    game_data['chrf difference'].append(chrf_after - chrf_before)

    game_data['cos_sim (before)'].append(dataset_splits['test']['Cos_sim_t5'][i])

    # response_embeddings, target_embeddings = con_sent_emb(game_data["response (after)"][i], target)
    cos_sim = cosine_similarity(con_sent_emb(game_data['response (after)'][i],target))[0][1]
    game_data['cos_sim (after)'].append(cos_sim)
    game_data['cos_sim difference'].append(cos_sim - dataset_splits['test']['Cos_sim_t5'][i])

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results.to_csv('results.csv')
df_results

In [ ]:
import os
os.makedirs("rl_model", exist_ok=True) # Create the 'model' directory if it doesn't exist

model.save_pretrained("rl_model")

# ppo_trainer.save("rl_model/test_rl_model_t5_small")

In [ ]:
!zip -r rl_file.zip rl_model

# from google.colab import files
# files.download("/content/rl_file.zip")